In [1]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install tflearn


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import nltk
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("omw-1.4")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nakshatra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Nakshatra\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Nakshatra\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:

from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# Importing Libraries needed for Tensorflow processing
import tensorflow as tf   #version 1.13.2
import numpy as np
import tflearn            #version 0.3.2
import random
import json

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [4]:
# importing our intent file used for training the model.
with open("intents.json") as json_data: 
    intents = json.load(json_data)  

In [5]:
# Empty lists for appending the data after processing NLP
words=[]
documents = []
classes = []


# This list will be used for ignoring all unwanted punctuation marks.
ignore = ["?",'.',',','!']

# Starting a loop through each intent in intents["patterns"]
for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        
        # tokenizing each and every word in the sentence by using word tokenizer and storing in w
        w = nltk.word_tokenize(pattern) 
        #print(w)
        
        # Adding tokenized words to words empty list that we created
        words.extend(w) 
        #print(words)
        
        # Adding words to documents with tag given in intents file
        documents.append((w, intent["tag"]))
        #print(documents)
        
        # Adding only tag to our classes list
        if intent["tag"] not in classes:      
            classes.append(intent["tag"])  #If tag is not present in classes[] then it will append into it.
            #print(classes)

In [6]:
#Performing Stemming by using stemmer.stem() nd lower each word 
#Running loop in words[] and ignoring punctuation marks present in ignore[]

words = [stemmer.stem(w.lower()) for w in words if w not in ignore]  
words = sorted(list(set(words)))  #Removing Duplicates in words[]

#Removing Duplicate Classes
classes = sorted(list(set(classes)))

#Printing length of lists we formed
print(len(documents),"Documents \n")
print(len(classes),"Classes \n")
print(len(words), "Stemmed Words ")


104 Documents 

17 Classes 

65 Stemmed Words 


In [7]:
#Creating Training Data which will be furthur used for training
training = []
output = []

#Creating empty array for output
output_empty = [0] * len(classes)

#Creating Training set and bag of words for each sentence
for doc in documents:
    bag = [] #Initialising empty bag of words

    pattern_words = doc[0] #Storing list of tokenized words for the documents[] tp pattern_words
    #print(pattern_words)
    
    #Again Stemming each word from pattern_words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]  
    #print(pattern_words)
    
    #Creating bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
    #It will give output 1 for curent tag and 0 for all other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] =1 
    training.append([bag, output_row])


In [8]:
random.shuffle(training) #Suffling the data or features
training = np.array(training) #Converting training data into numpy array

#Creating Training Lists
train_x = list(training[:,0])
train_y = list(training[:,1])



C:\Users\Nakshatra\AppData\Local\Temp\ipykernel_14304\1336731122.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training) #Converting training data into numpy array


In [9]:
from tensorflow.python.framework import ops
#ops.reset_default_graph()

ops.reset_default_graph() #Reset Underlying Graph data

#Building our own Neural Network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation="softmax")
net = tflearn.regression(net)

#Defining Model and setting up tensorboard
model = tflearn.DNN(net, tensorboard_dir="tflearn_logs") 

#Now we have setup model, now we need to train that model by fitting data into it by model.fit()
#n_epoch is the number of times that model will se our data during training
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True) 
model.save("model.tflearn") #Saving the model



Training Step: 12999  | total loss: 0.00153 | time: 0.024s
| Adam | epoch: 1000 | loss: 0.00153 - acc: 1.0000 -- iter: 096/104
Training Step: 13000  | total loss: 0.00154 | time: 0.024s
| Adam | epoch: 1000 | loss: 0.00154 - acc: 1.0000 -- iter: 104/104
--
INFO:tensorflow:C:\Users\Nakshatra\Desktop\chatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [10]:
#Importing pickle module
import pickle

#Dumping training data by using dump() and writing it into training_data in binary mode
pickle.dump({"words":words, "classes":classes, "train_x":train_x, "train_y":train_y}, open("training_data", "wb"))


In [11]:
#Restoring all data structure
data = pickle.load(open("training_data","rb"))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


In [12]:
with open("intents.json") as json_data:
    intents = json.load(json_data)  #Loading our json_data


In [13]:
# Loading the saved model
model.load("./model.tflearn") #Loading training model which we saved


INFO:tensorflow:Restoring parameters from C:\Users\Nakshatra\Desktop\chatbot\model.tflearn


In [14]:
#Cleaning User Input
def clean_up_sentence(sentence):
    
    # Tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence) #Again tokenizing the sentence
    
    #Stemming each word from the user's input
    sentence_words= [stemmer.stem(word.lower()) for word in sentence_words]

    return sentence_words

#Returning bag of words array: 0 or 1 or each word in the bag that exists in as we have declared in above lines
def bow(sentence, words, show_details=False):
    
    #Tokenizing the user input
    sentence_words = clean_up_sentence(sentence)
    
    #Generating bag of words from the sentence that user entered
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("Found in bag: %s"% w)
    return(np.array(bag))

In [15]:
#Adding some context to the conversation for better results.

context = {} #Create a dictionary to hold user's context

ERROR_THRESHOLD = 0.25
def classify(sentence):
    
    #Generating probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    
    #Filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    
    #Sorting by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    
    #If we have a classification then find the matching intent tag
    if results:
        
        #Loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                
                #Find a tag matching the first result
                if i['tag'] == results[0][0]:
                    
                    #Set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']
                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        
                        #A random response from the intent
                        return random.choice(i['responses'])

            results.pop(0)


In [23]:
print("Go..Bot is Running!!!")
while True:
    message = input("")
    if message=="exit":
        break
    ints = classify(message)
    res = response(message,intents)


Go..Bot is Running!!!
Hello
Hello.. Welcome to Shoe Tex. We have a variety of shoes available.May I know Your Preferences
tell me more
We offer collections of 
1.Sneakers
2.Boots
3.Slippers
4.Sports.
Enter your preference to get more details.
Sports
Flexible and Comfortable styles for Sports Shoes
1.High Ankle Knitwear Running Shoes for Men
2.Men's Wonder-13 Sports Running Shoes
3.Men's Rugged Trekking Shoes 1225
4.Women Walking Shoes
5.Slip on Running Sports Shoes
If you want to filter search for men/women Type
* Sport shoes for Men - for men's Slippers
* Sports shoes for Women - for women's Slippers
exit


In [24]:
from tkinter import *
bot_name = "ChatBot"
import jarvis_edit as js

BG_GRAY = "#FFD700"
BG_COLOR = "#FFFFCB"
TEXT_COLOR = "#000000"
SEND_COLOR = '#F0FFFF'

FONT = "Helvetica 14"
FONT_BOLD = "Helvetica 13 bold"


class ChatApplication:
    
    def __init__(self):
        self.window = Tk()
        self._setup_main_window()
        
    def run(self):
        self.window.mainloop()
        
    def _setup_main_window(self):
        self.window.title("Shoe Tex ChatBot")
        self.window.resizable(width = True,height=True)
        self.window.configure(width=600,height=700,bg= BG_COLOR)
        
        #head label
        head_label = Label(self.window,bg=BG_GRAY,fg=TEXT_COLOR,
                          text="WELCOME...",font = FONT_BOLD,pady=10)
        head_label.place(relwidth=1)
        
        #tiny dicider
        line = Label(self.window,width = 450,bg = BG_COLOR)
        line.place(relwidth=1,rely=0.07,relheight=0.012)
        
        #text widget
        self.text_widget = Text(self.window,width=20,height=2,bg=TEXT_COLOR,fg = SEND_COLOR,font=FONT,padx=5,pady=5)
        self.text_widget.place(relheight=0.745,relwidth=1,rely=0.08)
        self.text_widget.configure(cursor="arrow",state=DISABLED)
        
        #scrollbar
        scrollbar = Scrollbar(self.text_widget)
        scrollbar.place(relheight=1,relx=0.975)
        scrollbar.configure(command=self.text_widget.yview)
        
        #bottom label
        bottom_label = Label(self.window,bg = BG_GRAY,height=80)
        bottom_label.place(relwidth=1,rely=0.825)
        
        #text Entry widget
        self.msg_entry = Entry(bottom_label,bg=BG_COLOR,fg=TEXT_COLOR,font=FONT)
        self.msg_entry.place(relwidth=0.74,relheight=0.06,rely=0.008,relx=0.011)
        self.msg_entry.focus()
        self.msg_entry.bind("<Return>",self._on_enter_pressed)
    
        #send button
        send_button = Button(bottom_label,text="Send",font=FONT_BOLD,width=20,bg=BG_GRAY,
                        command=lambda:self._on_enter_pressed(None))
        send_button.place(relx=0.77,rely=0.008,relheight=0.06,relwidth=0.22)
        
    def _on_enter_pressed(self,event):
        msg = self.msg_entry.get()
        self._insert_message(msg,"You:")
        
    
        
        
    def _insert_message(self,msg,sender):
        if not msg:
            return
        self.msg_entry.delete(0,END)
        #js.takeCommand()
        msg1 = f"{sender}: {msg}\n\n"
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END,msg1)
        self.text_widget.configure(state=DISABLED)
        res = response(msg,intents)
        
        msg2 = f"{bot_name}: {res}\n\n"
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END,msg2)
        self.text_widget.configure(state=DISABLED)
        
        self.text_widget.see(END)
        js.speak(res)
        
        
if __name__=="__main__":
    js.wish_me()
    app = ChatApplication()
    app.run()